In [ ]:
IDENTIFIER = "haydn-op1"
NUM = 3

import json
import networkx as nx
import pandas as pd
from dwave.system import DWaveSampler, FixedEmbeddingComposite

from plots import plotCSV
from qubo import createBQM
from samples import totalEntropy


picklePath = f"../Pickles/{IDENTIFIER}/{IDENTIFIER}_"
numPath = f"../Pickles/{IDENTIFIER}/{NUM}/{IDENTIFIER}_{NUM}_"
dataPath = f"../Data/{IDENTIFIER}/{NUM}/{IDENTIFIER}_{NUM}_"

phrases = pd.read_csv(picklePath + "phrases.csv", index_col=[0,1])
G = nx.read_graphml(picklePath + "graph.graphml")
instruments = json.load(open(numPath + "instruments.json"))
embedding = json.load(open(numPath + "embedding.json", "r"))
qa = FixedEmbeddingComposite(DWaveSampler(), embedding)

In [ ]:
cs = 25

def recordAnnealTime(annealRange, bqm, filepath: str):
    
    file = pd.DataFrame()
    reads = 500

    for a in annealRange:
        sampleset = qa.sample(bqm, num_reads=reads, chain_strength=cs, annealing_time=a, label=f"{IDENTIFIER}_{NUM}_at={a}")
        sample = sampleset.first

        new_row = pd.DataFrame({
            "Anneal time": [sampleset.info["timing"]["qpu_anneal_time_per_sample"]],
            "QPU time": [sampleset.info["timing"]["qpu_sampling_time"]],
            "Chain strength": [cs],
            "Total reads": [reads],
            "Chain break fraction": [sample.chain_break_fraction],
            "Lowest energy": [bqm.energy(sample.sample)],
            "Total entropy": [totalEntropy(sample.sample, G)]
            })
    
        print(f"Anneal time {a} us recorded!")
        file = pd.concat([file, new_row])

    file.to_csv(filepath, index=False, mode="a", header=False)

# 10–300 repeated 3 times
annealTimes = range(10,310,10)
bqm = createBQM(G, phrases, instruments, 6, 6)

for _ in range(2):
    recordAnnealTime(annealTimes, bqm, dataPath+"anneal-time.csv")